Mount google drive for google colab

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

input_folder = '/content/drive/MyDrive/Colab Notebooks/fyp/images/noise_data/4_noise_img'
output_folder = '/content/drive/MyDrive/Colab Notebooks/fyp/images/noise_data/16_noise_img'

if not os.path.exists(input_folder):
    print(f"Input folder '{input_folder}' not found.")
else:
    print(f"Input folder '{input_folder}' found.")

if not os.path.exists(output_folder):
    print(f"Output folder '{output_folder}' not found.")
else:
    print(f"Output folder '{output_folder}' found.")


Mounted at /content/drive
Input folder '/content/drive/MyDrive/Colab Notebooks/fyp/images/noise_data/4_noise_img' found.
Output folder '/content/drive/MyDrive/Colab Notebooks/fyp/images/noise_data/16_noise_img' found.


U-Net model for perfect data.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint

# function to load and preprocess images
def load_and_preprocess_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Resize images to (128, 128)
            img = img.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
            images.append(img)
    return np.array(images)

# folders for low resolution (4 sources and detectors) and high resolution (16 sources and detectors) images
low_res_folder = '/content/drive/MyDrive/Colab Notebooks/fyp/images/noise_data/4_noise_img'
high_res_folder = '/content/drive/MyDrive/Colab Notebooks/fyp/images/noise_data/16_noise_img'

# Load and preprocess low resolution images
low_res_images = load_and_preprocess_images(low_res_folder)

# Load and preprocess high resolution images
high_res_images = load_and_preprocess_images(high_res_folder)

# split data into training and testing sets
train_low_images, test_low_images, train_high_images, test_high_images = train_test_split(
    low_res_images, high_res_images, test_size=0.2, random_state=42
)

# U-Net model architecture
def unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder
    up6 = layers.Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(conv5)
    up6 = layers.concatenate([up6, conv4], axis=3)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    up7 = layers.concatenate([up7, conv3], axis=3)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    up8 = layers.concatenate([up8, conv2], axis=3)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    up9 = layers.concatenate([up9, conv1], axis=3)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv9) 

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="img_to_img_unet")
    return model


# build U-Net model
input_shape = (128, 128, 1)  # Adjust input shape based on your data
model = unet(input_shape)

# define the optimiser with a custom learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss='mean_squared_error')

# define a ModelCheckpoint callback to save the model during training
checkpoint_path = 'trained_unet_img_to_img_model.h5'
model_checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True)

model.fit(train_low_images, train_high_images,
          epochs=200, batch_size=32, validation_split=0.2, callbacks=[model_checkpoint])


# fefine a ModelCheckpoint callback to save the model during training
checkpoint_path = 'trained_unet_img_to_img_model.h5'
model_checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True)

model.evaluate(test_low_images, test_high_images)

model.save('trained_unet_img_to_img_model.h5')


U-Net for Noise data.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint

# define function to load and preprocess images
def load_and_preprocess_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 128))  # resize images to (128, 128)
            img = img.astype('float32') / 255.0  # normalize pixel values to [0, 1]
            images.append(img)
    return np.array(images)

# define U-Net model
def img_unet(input_shape, num_filters, kernel_size, optimizer, learning_rate):
    inputs = tf.keras.Input(shape=input_shape)

    # encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    pool5 = layers.MaxPooling2D(pool_size=(2, 2))(conv5)

    # bottleneck
    conv6 = layers.Conv2D(2048, 3, activation='relu', padding='same')(pool5)
    conv6 = layers.Conv2D(2048, 3, activation='relu', padding='same')(conv6)

    # decoder
    up7 = layers.Conv2DTranspose(1024, 2, strides=(2, 2), padding='same')(conv6)
    up7 = layers.concatenate([up7, conv5], axis=3)
    conv7 = layers.Conv2D(1024, 3, activation='relu', padding='same')(up7)
    conv7 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv7)

    up8 = layers.Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(conv7)
    up8 = layers.concatenate([up8, conv4], axis=3)
    conv8 = layers.Conv2D(512, 3, activation='relu', padding='same')(up8)
    conv8 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv8)

    up9 = layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv8)
    up9 = layers.concatenate([up9, conv3], axis=3)
    conv9 = layers.Conv2D(256, 3, activation='relu', padding='same')(up9)
    conv9 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv9)

    up10 = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv9)
    up10 = layers.concatenate([up10, conv2], axis=3)
    conv10 = layers.Conv2D(128, 3, activation='relu', padding='same')(up10)
    conv10 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv10)

    up11 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv10)
    up11 = layers.concatenate([up11, conv1], axis=3)
    conv11 = layers.Conv2D(64, 3, activation='relu', padding='same')(up11)
    conv11 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv11)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv11)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="img_unet_complex")
    return model

low_res_folder = '/content/drive/MyDrive/Colab Notebooks/fyp/images/noise_data/4_noise_img'
high_res_folder = '/content/drive/MyDrive/Colab Notebooks/fyp/images/noise_data/16_noise_img'

# load and preprocess low resolution images
low_res_images = load_and_preprocess_images(low_res_folder)

# load and preprocess high resolution images
high_res_images = load_and_preprocess_images(high_res_folder)

# split data into training and testing sets
train_low_images, test_low_images, train_high_images, test_high_images = train_test_split(
    low_res_images, high_res_images, test_size=0.2, random_state=42
)

learning_rate = 0.001
num_filters = 64
kernel_size = 5
optimizer = 'adam'
input_shape = (128, 128, 1)
model = img_unet(input_shape=input_shape, num_filters=num_filters, kernel_size=kernel_size,
                 optimizer=optimizer, learning_rate=learning_rate)

# compile the model
model.compile(optimizer=optimizer, loss='mean_squared_error')

# define a ModelCheckpoint callback to save the model during training
checkpoint_path = 'trained_unet_img_to_img_model.h5'
model_checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True)

# train the model
model.fit(train_low_images, train_high_images,
          epochs=150, batch_size=32, validation_split=0.2, callbacks=[model_checkpoint])

model.evaluate(test_low_images, test_high_images)

model.save('trained_unet_img_to_img_model.h5')


Testing my model on new data.

In [ ]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# load the trained model
model = load_model("trained_unet_img_to_img_model.h5")

# define function to load and preprocess a single image
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is not None:
        image = cv2.resize(image, (128, 128))  # Resize image to (128, 128)
        image = image.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
    return image  

input_folder = '/content/drive/MyDrive/Colab Notebooks/fyp/images/noise_data/4_noise_img'
output_folder = '/content/drive/MyDrive/Colab Notebooks/fyp/images/noise_data/16_noise_img'

image_filenames = os.listdir(input_folder)

num_images_to_test = 8  
random_image_filenames = random.sample(image_filenames, num_images_to_test)

# initialize list to store images
images = []

for random_image_filename in random_image_filenames:
    input_image_path = os.path.join(input_folder, random_image_filename)
    input_image = load_and_preprocess_image(input_image_path)

    # skip processing if image loading fails
    if input_image is None:
        print(f"Skipping image: {random_image_filename} (failed to load)")
        continue

    # reshape input image for model prediction
    input_image = np.expand_dims(input_image, axis=0)

    # predict the output image using the trained model
    output_image = model.predict(input_image)

    # load the corresponding ground truth output image
    ground_truth_image_path = os.path.join(output_folder, random_image_filename)
    ground_truth_image = load_and_preprocess_image(ground_truth_image_path)

    # skip processing if ground truth image loading fails
    if ground_truth_image is None:
        print(f"Skipping image: {random_image_filename} (ground truth image failed to load)")
        continue

    plt.figure(figsize=(10, 5))

    # original input image
    plt.subplot(1, 3, 1)
    plt.imshow(input_image[0], cmap='gray')
    plt.title('Original Input Image')
    plt.axis('off')

    # predicted output image
    plt.subplot(1, 3, 2)
    plt.imshow(output_image[0, :, :, 0], cmap='gray')
    plt.title('Predicted Output Image')
    plt.axis('off')

    # ground truth output image
    plt.subplot(1, 3, 3)
    plt.imshow(ground_truth_image, cmap='gray')
    plt.title('Ground Truth Output Image')
    plt.axis('off')

    plt.show()

    images.append(np.concatenate([input_image[0], output_image[0, :, :, 0], ground_truth_image], axis=1))

grid_image = np.concatenate(images, axis=0)

cv2.imwrite("grid_image.png", grid_image)

plt.imshow(grid_image, cmap='gray')
plt.axis('off')
plt.show()


Accuracy and loss graphs.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import os
import cv2

def load_image_samples(input_folder, output_folder):
    image_samples = []

    input_filenames = sorted(os.listdir(input_folder))
    output_filenames = sorted(os.listdir(output_folder))

    for input_filename, output_filename in zip(input_filenames, output_filenames):
        input_img_path = os.path.join(input_folder, input_filename)
        output_img_path = os.path.join(output_folder, output_filename)

        input_img = cv2.imread(input_img_path, cv2.IMREAD_GRAYSCALE)
        output_img = cv2.imread(output_img_path, cv2.IMREAD_GRAYSCALE)

        if input_img is not None and output_img is not None:
            input_img = cv2.resize(input_img, (128, 128))  
            output_img = cv2.resize(output_img, (128, 128))  

            input_img = input_img.astype('float32') / 255.0 
            output_img = output_img.astype('float32') / 255.0 

            image_samples.append((input_img, output_img))

    return image_samples

input_folder = '/jupyter/work/fyp/data/img_rec/4th_set_noise/4_noise'
output_folder = '/jupyter/work/fyp/data/img_rec/4th_set_noise/16_noise'
image_samples = load_image_samples(input_folder, output_folder)


def load_ground_truth_images(folder_path):
    ground_truth_images = []

    filenames = sorted(os.listdir(folder_path))

    for filename in filenames:
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is not None:
            img = cv2.resize(img, (128, 128)) 
            img = img.astype('float32') / 255.0  
            ground_truth_images.append(img)

    return ground_truth_images

ground_truth_folder = '/jupyter/work/fyp/data/img_rec/4th_set_noise/16_noise'
reconstructions = '/jupyter/work/fyp/code/results/img'
ground_truth_images = load_ground_truth_images(ground_truth_folder)


# variables needed for analysis and plotting
train_losses = history.history['loss']
val_losses = history.history['val_loss']
train_accs = history.history['accuracy']  
val_accs = history.history['val_accuracy'] 
reconstructed_images = load_ground_truth_images(reconstructions)

# Loss Curves
def plot_loss_curves(train_losses, val_losses, task_name, save_dir):
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title(f'{task_name} Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(save_dir, f'{task_name}_loss_curves.png'))
    plt.show()

# # Accuracy Curves
def plot_accuracy_curves(train_accs, val_accs, task_name, save_dir):
    plt.plot(train_accs, label='Training Accuracy')
    plt.plot(val_accs, label='Validation Accuracy')
    plt.title(f'{task_name} Accuracy Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(os.path.join(save_dir, f'{task_name}_accuracy_curves.png'))
    plt.show()

results_folder = '/jupyter/work/fyp/code/results'

# Now you can call the plotting and analysis functions with the defined variables
plot_loss_curves(train_losses, val_losses, 'Image Reconstruction', results_folder)
plot_accuracy_curves(train_accs, val_accs, 'Image Reconstruction', results_folder)



PSNR, SSIM, MSE variation graphs.

In [ ]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio, mean_squared_error, structural_similarity
from tensorflow.keras.models import load_model

# PSNR loss function
def psnr_loss(y_true, y_pred):
    return -K.mean(10.0 * K.log(K.square(1.0) / (K.square(y_pred - y_true) + K.epsilon())) / K.log(10.0))

model = load_model("unet_optimum_parameters.h5", custom_objects={'psnr_loss': psnr_loss})

# function to load and preprocess a single image
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is not None:
        image = cv2.resize(image, (128, 128)) 
        image = image.astype('float32') / 255.0 
    return image

input_folder = '/jupyter/work/fyp/data/img_rec/3rd_set/4'
output_folder = '/jupyter/work/fyp/data/img_rec/3rd_set/16'

image_filenames = os.listdir(input_folder)

num_images_to_test = 10 
random_image_filenames = random.sample(image_filenames, num_images_to_test)

psnr_scores = []
mse_scores = []
ssim_scores = []

for random_image_filename in random_image_filenames:
    input_image_path = os.path.join(input_folder, random_image_filename)
    input_image = load_and_preprocess_image(input_image_path)

    # reshape input image for model prediction
    input_image = np.expand_dims(input_image, axis=0)

    # predict the output image using the trained model
    output_image = model.predict(input_image)

    ground_truth_image_path = os.path.join(output_folder, random_image_filename)
    ground_truth_image = load_and_preprocess_image(ground_truth_image_path)

    # calculate PSNR (Peak Signal-to-Noise Ratio)
    psnr_value = peak_signal_noise_ratio(ground_truth_image, output_image[0, :, :, 0])

    # calculate MSE (Mean Squared Error)
    mse_value = mean_squared_error(ground_truth_image, output_image[0, :, :, 0])

    # calculate SSIM (Structural Similarity Index)
    ssim_value = structural_similarity(ground_truth_image, output_image[0, :, :, 0], data_range=1.0)

    # # Print metrics values
    # print(f"Metrics for {random_image_filename}:")
    # print(f"PSNR: {psnr_value}")
    # print(f"MSE: {mse_value}")
    # print(f"SSIM: {ssim_value}\n")

    psnr_scores.append(psnr_value)
    mse_scores.append(mse_value)
    ssim_scores.append(ssim_value)

# Plot PSNR, MSE, and SSIM scores
plt.figure(figsize=(10, 6))

plt.subplot(3, 1, 1)
plt.plot(psnr_scores, marker='o')
plt.title('PSNR Scores')
plt.xlabel('Image Index')
plt.ylabel('PSNR')

plt.subplot(3, 1, 2)
plt.plot(mse_scores, marker='o')
plt.title('MSE Scores')
plt.xlabel('Image Index')
plt.ylabel('MSE')

plt.subplot(3, 1, 3)
plt.plot(ssim_scores, marker='o')
plt.title('SSIM Scores')
plt.xlabel('Image Index')
plt.ylabel('SSIM')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# assuming psnr_scores, mse_scores, and ssim_scores are lists of scores
# calculate histogram for PSNR scores
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.hist(psnr_scores, bins=20, color='blue', alpha=0.7)
plt.title('PSNR Distribution')
plt.xlabel('PSNR Score')
plt.ylabel('Frequency')

# calculate histogram for MSE scores
plt.subplot(1, 3, 2)
plt.hist(mse_scores, bins=20, color='pink', alpha=0.7)
plt.title('MSE Distribution')
plt.xlabel('MSE Score')
plt.ylabel('Frequency')

# calculate histogram for SSIM scores
plt.subplot(1, 3, 3)
plt.hist(ssim_scores, bins=20, color='purple', alpha=0.7)
plt.title('SSIM Distribution')
plt.xlabel('SSIM Score')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# calculate percentages of values within certain ranges for each metric
def calculate_percentage(scores, range_start, range_end):
    values_within_range = [score for score in scores if range_start <= score <= range_end]
    percentage = (len(values_within_range) / len(scores)) * 100
    return percentage

# define range for each metric
psnr_range = (30, 40)
mse_range = (0, 0.05)
ssim_range = (0.95, 1)

# calculate percentages for each range
psnr_percentage = calculate_percentage(psnr_scores, psnr_range[0], psnr_range[1])
mse_percentage = calculate_percentage(mse_scores, mse_range[0], mse_range[1])
ssim_percentage = calculate_percentage(ssim_scores, ssim_range[0], ssim_range[1])

print(f'Percentage of PSNR scores in range {psnr_range}: {psnr_percentage}%')
print(f'Percentage of MSE scores in range {mse_range}: {mse_percentage}%')
print(f'Percentage of SSIM scores in range {ssim_range}: {ssim_percentage}%')
